# Amazon Bedrock AgentCore Runtime에서 Strands Agent를 사용한 스트리밍 응답

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore Runtime을 사용하여 스트리밍 응답을 구현하는 방법을 배웁니다. 이 예제는 결과가 생성되는 대로 부분적으로 스트리밍하여 대량의 콘텐츠를 생성하거나 상당한 처리 시간이 필요한 작업에 대해 더 반응성 높은 사용자 경험을 제공하는 방법을 보여줍니다.


### 튜토리얼 세부 정보

|정보| 세부사항|
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 스트리밍을 사용한 대화형|
| Agent 유형          | 단일         |
| Agentic Framework   | Strands Agents |
| LLM model           | Anthropic Claude Haiku 4.5 |
| 튜토리얼 구성 요소 | AgentCore Runtime, Strands Agent 및 Amazon Bedrock Model을 사용한 스트리밍 응답 |
| 튜토리얼 분야   | 범용                                                                   |
| 예제 복잡도  | 쉬움                                                                             |
| 사용된 SDK            | Amazon BedrockAgentCore Python SDK 및 boto3|

### 튜토리얼 아키텍처

이 튜토리얼에서는 스트리밍 agent를 AgentCore runtime에 배포하는 방법을 설명합니다. 

데모 목적으로 스트리밍 기능을 갖춘 Amazon Bedrock model을 사용하는 Strands Agent를 사용합니다.

예제에서는 `get_weather`와 `get_time`이라는 두 가지 tool을 가진 간단한 agent를 사용하지만 스트리밍 응답 기능을 갖추고 있습니다.

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore Runtime에서 agent의 스트리밍 응답
* 실시간 부분 결과 전달
* 스트리밍을 사용한 Amazon Bedrock model 사용
* 비동기 스트리밍 지원을 사용한 Strands Agent 사용

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker 실행 중

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## AgentCore Runtime에 배포할 스트리밍 agent 준비

이제 스트리밍 agent를 AgentCore Runtime에 배포해 보겠습니다. 스트리밍 기능은 entrypoint 함수에서 비동기 generator 또는 yield 문을 사용할 때 AgentCore SDK에 의해 자동으로 처리됩니다.

스트리밍 구현의 핵심 사항:
* entrypoint 함수에 `async def` 사용
* 청크가 사용 가능해지는 대로 스트리밍하기 위해 `yield` 사용
* AgentCore SDK가 Server-Sent Events (SSE) 형식을 자동으로 처리
* 클라이언트는 Content-Type: text/event-stream 응답을 받음

### Amazon Bedrock model 및 스트리밍을 사용한 Strands Agent
Amazon Bedrock model을 사용하는 Strands Agent의 스트리밍 구현을 살펴보겠습니다.

In [ ]:
%%writefile strands_claude_streaming.py
from strands import Agent, tool
from strands_tools import calculator
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# 커스텀 tool 정의
@tool
def weather():
    """ Get weather """
    return "sunny"  # 데모용 더미 구현

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time."""
)

@app.entrypoint
async def strands_agent_bedrock_streaming(payload):
    """
    Invoke the agent with streaming capabilities
    This function demonstrates how to implement streaming responses
    with AgentCore Runtime using async generators
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    
    try:
        # agent.stream_async()는 비동기 generator를 반환하여 실시간 스트리밍 가능
        async for event in agent.stream_async(user_input):
            if "data" in event:
                yield event["data"]  # yield로 청크 단위 스트리밍
            
    except Exception as e:
        # 스트리밍 중 에러 발생 시 에러 응답도 스트림으로 전송
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"Streaming error: {error_response}")
        yield error_response

if __name__ == "__main__":
    app.run()

## AgentCore Runtime의 스트리밍 이해

AgentCore Runtime에서 스트리밍을 사용하면 여러 가지가 자동으로 발생합니다:

### Server-Sent Events (SSE) 형식
* AgentCore SDK가 yield된 데이터를 자동으로 SSE 형식으로 변환
* 각 yield는 SSE 스트림에서 `data: ` 이벤트가 됨
* Content-Type이 자동으로 `text/event-stream`으로 설정됨

### 클라이언트 처리
* 클라이언트는 agent가 요청을 처리하는 동안 실시간 업데이트를 받음
* 이를 통해 점진적 응답 표시 및 더 나은 사용자 경험 제공
* 클라이언트는 전체 응답이 준비되기 전에 부분 결과를 처리할 수 있음

### 오류 처리
* 스트리밍 응답에는 적절한 오류 처리가 포함되어야 함
* 오류는 스트림의 일부로 yield될 수 있음
* 함수가 완료되거나 처리되지 않은 예외가 발생하면 스트림이 종료됨

## AgentCore Runtime에 스트리밍 agent 배포

`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성하여 클라이언트가 agent와 통신하는 방법을 제어할 수 있습니다. 

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다

이 튜토리얼에서는 Amazon Bedrock AgentCode Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore runtime에 배포합니다.

### AgentCore Runtime deployment 구성

다음으로 스타터 툴킷을 사용하여 entrypoint, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore Runtime deployment를 구성합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 스타터 킷을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

# AgentCore Runtime 구성: entrypoint, 실행 역할, ECR 자동 생성 설정
response = agentcore_runtime.configure(
    entrypoint="strands_claude_streaming.py",
    auto_create_execution_role=True,  # 실행 역할 자동 생성
    auto_create_ecr=True,  # ECR 리포지토리 자동 생성
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_streaming"
)
response

### AgentCore Runtime에 스트리밍 agent 시작

이제 docker 파일이 준비되었으므로 스트리밍 agent를 AgentCore Runtime에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore Runtime이 생성됩니다

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인해 보겠습니다

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
# Runtime이 준비 상태가 될 때까지 폴링
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### 스트리밍으로 AgentCore Runtime 호출

마지막으로 페이로드로 AgentCore Runtime을 호출하고 스트리밍 응답을 받을 수 있습니다

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({
    "prompt": 
    "what the weather is like?"
})
invoke_response

### 스트리밍을 위해 boto3로 AgentCore Runtime 호출

이제 AgentCore Runtime이 생성되었으므로 모든 AWS SDK로 호출할 수 있습니다. 스트리밍 응답의 경우 Server-Sent Events 형식을 처리해야 합니다.

In [ ]:
import boto3
import json
from IPython.display import Markdown, display

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# 스트리밍 응답을 위해 EventStream 처리 필요
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2+1"})
)

# Content-Type이 text/event-stream인지 확인하여 스트리밍 응답 처리
if "text/event-stream" in boto3_response.get("contentType", ""):
    print("Processing streaming response with boto3:")
    content = []
    # SSE 형식으로 전송되는 각 라인을 파싱
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                data = line[6:].replace('"', '')  # "data: " prefix 제거
                print(f"Received streaming chunk: {data}")
                content.append(data.replace('"', ''))
    
    # 스트리밍된 전체 응답 표시
    full_response = " ".join(content)
    display(Markdown(full_response))
else:
    # 비스트리밍 응답 처리
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    
    if events:
        try:
            response_data = json.loads(events[0].decode("utf-8"))
            display(Markdown(response_data))
        except:
            print(f"Raw response: {events[0]}")

## 스트리밍 응답의 이점

스트리밍 응답은 여러 가지 주요 이점을 제공합니다:

### 사용자 경험
* **즉각적인 피드백**: 사용자는 결과가 사용 가능해지는 대로 부분 결과를 확인
* **체감 성능**: 전체 시간이 동일하더라도 응답이 더 빠르게 느껴짐
* **점진적 표시**: 긴 응답을 점진적으로 표시할 수 있음

### 기술적 이점
* **메모리 효율성**: 모든 것을 메모리에 로드하지 않고 대용량 응답 처리
* **타임아웃 방지**: 장시간 실행되는 작업에서 타임아웃 방지
* **실시간 처리**: 실시간 데이터를 사용 가능해지는 대로 처리

### 사용 사례
* **콘텐츠 생성**: 장문 작성, 보고서, 문서
* **데이터 분석**: 복잡한 계산의 점진적 결과
* **다단계 워크플로**: 복잡한 agent 추론을 통한 진행 상황 표시
* **실시간 모니터링**: 모니터링 agent의 실시간 업데이트

## 정리 (선택 사항)

이제 생성된 AgentCore Runtime을 정리해 보겠습니다

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

# AgentCore Runtime 삭제
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

# ECR 리포지토리 삭제 (force=True로 이미지 포함 전체 삭제)
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# 축하합니다!

Amazon Bedrock AgentCore Runtime을 사용하여 스트리밍 agent를 성공적으로 구현하고 배포했습니다! 

## 배운 내용:
* 비동기 generator를 사용하여 스트리밍 응답을 구현하는 방법
* AgentCore Runtime이 SSE 형식을 자동으로 처리하는 방법
* 클라이언트 측에서 스트리밍 응답을 처리하는 방법
* 사용자 경험 및 성능을 위한 스트리밍의 이점

## 다음 단계:
* 사용 사례에 맞는 다양한 스트리밍 패턴 실험
* 복잡한 다단계 워크플로를 위한 커스텀 스트리밍 로직 구현
* Memory 및 Gateway와 같은 다른 AgentCore 기능과 스트리밍 결합 탐색
* 더 나은 UX를 위한 클라이언트 측 스트리밍 시각화 구현 고려